In [2]:
import os
from pathlib import Path, PureWindowsPath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [3]:
path_cwd = os.path.dirname(os.getcwd())

In [4]:
path_cwd = Path(os.path.dirname(os.getcwd()))
path_cwd

PosixPath('/Users/williamearley/Documents/GitHub/Store-Sales')

In [5]:
path = path_cwd / 'data'
path

PosixPath('/Users/williamearley/Documents/GitHub/Store-Sales/data')

In [172]:
df_train = pd.read_csv(path / 'train.csv')
df_transaction = pd.read_csv(path / 'transactions.csv')
df_holidays = pd.read_csv(path / 'holidays_events.csv')
df_oil = pd.read_csv(path / 'oil.csv')
df_stores = pd.read_csv(path / 'stores.csv')
df_test = pd.read_csv(path / 'test.csv')

## Clean Data ##

In [163]:
# Will here

# Just renaming the price column of the oil df

new_oil = df_oil.copy()

new_oil.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)

In [164]:
new_oil.head(3)

,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97


In [165]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 43


In [166]:
new_oil['oil_price'] = new_oil['oil_price'].interpolate(method='index')

In [167]:
new_oil.head(3)

,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97


In [168]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 1


In [169]:
new_oil.at[0, 'oil_price'] = 93.14

In [170]:
new_oil.head(2)

,date,oil_price
0,2013-01-01,93.14
1,2013-01-02,93.14


In [171]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 0


In [142]:
oil_price_interpolated = oil_price.interpolate(method='index')

In [180]:
new_oil['date'] = pd.to_datetime(new_oil['date'])

In [182]:
df_train['date'] = pd.to_datetime(df_train['date'])

In [179]:
new_oil.head(2)

,date,oil_price
0,2013-01-01,93.14
1,2013-01-02,93.14


In [183]:
df_train = pd.merge(df_train, new_oil, on=['date'])

In [184]:
df_train.head(3)

,id,date,store_nbr,family,sales,onpromotion,oil_price
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,93.14


#### shorten date ####

In [185]:
def train_to_store_merge(train, store):
    df_train_store_merged = pd.merge(train, store, how='left', on='store_nbr')
    return df_train_store_merged

In [186]:
# train clean

df_train['date'] = pd.to_datetime(df_train['date'])
first_sale_date_per_store = df_train[df_train['sales'] > 0].groupby('store_nbr')['date'].min().reset_index()

# remove rows before stores were open. only do this to train
df_train_min_date = pd.merge(df_train, first_sale_date_per_store, on='store_nbr')
df_train_shortened = df_train_min_date[df_train_min_date['date_x'] >= df_train_min_date['date_y']] 
df_train_shortened = df_train_shortened.drop(['date_y'], axis=1)
df_train_shortened.rename(columns={'date_x':'date'}, inplace=True)
df_train_shortened = train_to_store_merge(df_train_shortened, df_stores)


In [187]:
# add dates when stores were temporarily closed

#### holidays feature eng ####

In [189]:
# holidays 

df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays_real = df_holidays[df_holidays['transferred']==False] 

#unique_holis2 = df_holidays_real['description'].drop_duplicates()
unique_holis2 = df_holidays_real[['description','locale']].drop_duplicates()
unique_holis_national = unique_holis2[unique_holis2['locale']=='National'].drop(['locale'],axis=1)
unique_holis_city = unique_holis2[unique_holis2['locale']=='Local'].drop(['locale'],axis=1)
unique_holis_state = unique_holis2[unique_holis2['locale']=='Regional'].drop(['locale'],axis=1)

national_holidays = df_holidays_real[df_holidays_real['locale']=='National'].loc[:,('date','description')]
local_holidays = df_holidays_real[df_holidays_real['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays = df_holidays_real[df_holidays_real['locale']=='Regional'].loc[:,('date','description','locale_name')]

df_train_summed_daily = df_train_shortened.groupby(['date','city','state']).agg({'onpromotion':'sum', 'sales':'sum', 'oil_price':'sum'}).reset_index()


In [192]:
df_holi.head(3)

,date,city,state,onpromotion,sales,price,Primer Grito de Independencia,Traslado Independencia de Guayaquil,Dia de Difuntos,Independencia de Cuenca,...,Independencia de Latacunga,Independencia de Ambato,Fundacion de Quito-1,Fundacion de Quito,Fundacion de Loja,Cantonizacion de Salinas,Fundacion de Guayaquil-1,Fundacion de Guayaquil,Traslado Fundacion de Guayaquil,Traslado Fundacion de Quito
0,2013-01-01,Salinas,Santa Elena,0,2511.618999,0.00,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2013-01-02,Ambato,Tungurahua,0,23136.469000,6147.24,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2013-01-02,Babahoyo,Los Rios,0,6637.560000,3073.62,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [193]:
df_holi = pd.merge(df_train_summed_daily, national_holidays, how='left', on='date')
df_holi = pd.merge(df_holi, state_holidays, how='left', left_on=['date', 'state'], right_on=['date','locale_name'])
df_holi = pd.merge(df_holi, local_holidays, how='left', left_on=['date', 'city'], right_on=['date','locale_name'])

df_holi = df_holi.drop(['locale_name_x','locale_name_y'],axis=1)
df_holi = df_holi.rename(columns = {'description_x':'national_holiday','description_y':'state_holiday','description':'city_holiday'})

In [194]:
# set boolean column for each unique holiday. still have dups

for holiday in unique_holis_national['description'].tolist():
    df_holi[holiday] = df_holi['national_holiday'] == holiday
for holiday in unique_holis_state['description'].tolist():
    df_holi[holiday] = df_holi['state_holiday'] == holiday
for holiday in unique_holis_city['description'].tolist():
    df_holi[holiday] = df_holi['city_holiday'] == holiday

df_holi = df_holi.drop(['national_holiday','state_holiday','city_holiday'] ,axis=1)

/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_10363/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_holi[holiday] = df_holi['city_holiday'] == holiday
/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_10363/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_holi[holiday] = df_holi['city_holiday'] == holiday
/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_10363/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling 

In [195]:
df_holi.head(2)

,date,city,state,onpromotion,sales,oil_price,Primer Grito de Independencia,Traslado Independencia de Guayaquil,Dia de Difuntos,Independencia de Cuenca,...,Independencia de Latacunga,Independencia de Ambato,Fundacion de Quito-1,Fundacion de Quito,Fundacion de Loja,Cantonizacion de Salinas,Fundacion de Guayaquil-1,Fundacion de Guayaquil,Traslado Fundacion de Guayaquil,Traslado Fundacion de Quito
0,2013-01-01,Salinas,Santa Elena,0,2511.618999,3073.62,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2013-01-02,Ambato,Tungurahua,0,23136.469000,6147.24,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [197]:
# combines duplicates rows into 1 rows where there is a true for each holiday that falls on the given date
unique_holis_list = list(unique_holis2['description'])
agg_func = {col: 'any' for col in unique_holis_list}

aggregated_df = df_holi.groupby(['date', 'sales', 'city', 'state', 'onpromotion', 'oil_price']).agg(agg_func).reset_index()


aggregated_df = pd.get_dummies(aggregated_df, columns=['city', 'state'], prefix=['city', 'state'])

In [201]:
aggregated_df.head(3)

,date,sales,onpromotion,oil_price,Fundacion de Manta,Provincializacion de Cotopaxi,Fundacion de Cuenca,Cantonizacion de Libertad,Cantonizacion de Riobamba,Cantonizacion del Puyo,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,2013-01-01,2511.618999,0,3073.62,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2013-01-02,2978.224000,0,3073.62,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,2013-01-02,4973.285000,0,3073.62,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [202]:
#OLS to determine impact of each holiday across all stores

In [203]:
# seasonality has big impact on predicted sales so need to add some variables to capture 
aggregated_df_ols = aggregated_df.copy()
aggregated_df_ols['dow'] = aggregated_df_ols['date'].dt.dayofweek
aggregated_df_ols['month'] = aggregated_df_ols['date'].dt.month
aggregated_df_ols['year'] = aggregated_df_ols['date'].dt.year
aggregated_df_ols = aggregated_df_ols.drop('date',axis=1)

In [204]:
#model seasonality with dummy vars... do i need this?
aggregated_df_ols_dummies =  pd.get_dummies(aggregated_df_ols, columns=['dow','month', 'year'], drop_first=True)

# Will Adding in Feature Engineering of Oil interpolated, lag_1, and rolling mean
### Added oil pricing up above so they would aggregate correctly.

In [205]:
# Create day of week feature
#df_family_aggregated['day_of_week'] = df_family_aggregated['date'].dt.dayofweek

# Lag sales by 1 to get previous day of sales value
aggregated_df_ols_dummies['lag_1'] = aggregated_df_ols_dummies['sales'].shift(1)

# Add rolling mean for 7-day window
aggregated_df_ols_dummies['rolling_mean'] = aggregated_df_ols_dummies['sales'].rolling(window=7).mean()

In [208]:
aggregated_df_ols_dummies.head(3)

,sales,onpromotion,oil_price,Fundacion de Manta,Provincializacion de Cotopaxi,Fundacion de Cuenca,Cantonizacion de Libertad,Cantonizacion de Riobamba,Cantonizacion del Puyo,Cantonizacion de Guaranda,...,month_9,month_10,month_11,month_12,year_2014,year_2015,year_2016,year_2017,lag_1,rolling_mean
0,2511.618999,0,3073.62,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
1,2978.224000,0,3073.62,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2511.618999,NaN
2,4973.285000,0,3073.62,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2978.224000,NaN


In [211]:
aggregated_df_ols_dummies.to_pickle('aggregated_df_ols_dummies.pkl')

# Further Modeling

In [17]:
X = aggregated_df_ols_dummies.drop('sales',axis=1)
# add constant for linear regression
X = sm.add_constant(X)
X = X.astype(int)
y=aggregated_df_ols_dummies['sales']

In [18]:
model_OLS = sm.OLS(y,X).fit()
model_summary = model_OLS.summary()

In [ ]:
aggregated_df_ols_dummies

In [19]:
# remove features that have p value >.10 
# ... no xmas day in trainset
pvalues = model_OLS.pvalues
alpha = .05
drop_columns = pvalues[pvalues>alpha]
drop_cols = list(drop_columns.index)
# manually keeping non holiday columns
#most date columns were significant except for these 3 months. still would like to keep so there arent holes in date
drop_cols.remove('month_3')
drop_cols.remove('month_7')
drop_cols.remove('month_9')
drop_cols.remove('month_10')
drop_cols.remove('month_11')
drop_cols.remove('city_Ambato')
drop_cols.remove('state_Tungurahua')
drop_cols.remove('state_Guayas')

In [20]:
#values2 = pd.DataFrame(pvalues).reset_index()
#pvalues2.rename(columns={'index':'holiday',0:'p'} , inplace=True)
#pvalues2

In [21]:
print(len(drop_cols), 'holidays droped out of', len(unique_holis2),'.', len(unique_holis2)-len(drop_cols) ,'unique holidays remain' )

81 holidays droped out of 103 . 22 unique holidays remain


In [22]:
df_holi_shortened = df_holidays_real[~df_holidays_real['description'].isin(drop_cols)]
df_holi_shortened = df_holi_shortened.drop(['type','locale','locale_name','transferred'],axis=1)

In [23]:
# unique holidays from the reduced holidays list
unique_holidays = df_holi_shortened['description'].unique()
filtered_holidays = df_holidays[df_holidays['description'].isin(unique_holidays)]

In [24]:
#separate these out because they have differenct merge conditions
national_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='National'].loc[:,('date','description')]
local_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Regional'].loc[:,('date','description','locale_name')]

In [25]:
# unique holidays from the reduced holidays list
unique_holidays = df_holi_shortened['description'].unique()

#### combine train to holidays ####

In [26]:
# unique holidays from the reduced holidays list
filtered_holidays = df_holidays[df_holidays['description'].isin(unique_holidays)]
#separate locales out because they have differenct merge conditions
national_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='National'].loc[:,('date','description')]
local_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Regional'].loc[:,('date','description','locale_name')]

In [27]:
def train_to_holiday_merge(train, national_holidays_filtered, state_holidays_filtered, local_holidays_filtered):
    train['date'] = pd.to_datetime(train['date'])
    df_train_filtered = pd.merge(train, national_holidays_filtered, how='left', on='date')
    df_train_filtered = pd.merge(df_train_filtered, state_holidays_filtered, how='left', left_on=['date', 'state'], right_on=['date','locale_name'])
    df_train_filtered = pd.merge(df_train_filtered, local_holidays_filtered, how='left', left_on=['date', 'city'], right_on=['date','locale_name'])
    df_train_filtered['holiday'] = df_train_filtered['description_x'].combine_first(df_train_filtered['description_y']).combine_first(df_train_filtered['description'])

    df_train_filtered = df_train_filtered.drop(['locale_name_x','locale_name_y','description','description_x','description_y'],axis=1)
    return df_train_filtered

In [28]:
df_train_filtered = train_to_holiday_merge(df_train_shortened, national_holidays_filtered, state_holidays_filtered,local_holidays_filtered)

#### combine oil and train ####

In [29]:
def train_to_oil_merge(train, oil):
    df_oil['date'] = pd.to_datetime(df_oil['date'])
    df = pd.merge(train,oil, how='left', on='date')
    return df

In [30]:
df_train_merged = train_to_oil_merge(df_train_filtered, df_oil)

df_train_merged = df_train_merged.drop(['id','city','state', 'type'], axis=1)

df_train_merged

,date,store_nbr,family,sales,onpromotion,cluster,holiday,dcoilwtico
0,2013-01-02,1,AUTOMOTIVE,2.000,0,13,NaN,93.14
1,2013-01-02,1,BABY CARE,0.000,0,13,NaN,93.14
2,2013-01-02,1,BEAUTY,2.000,0,13,NaN,93.14
3,2013-01-02,1,BEVERAGES,1091.000,0,13,NaN,93.14
4,2013-01-02,1,BOOKS,0.000,0,13,NaN,93.14
...,...,...,...,...,...,...,...,...
2778826,2017-08-15,9,POULTRY,438.133,0,6,NaN,47.57
2778827,2017-08-15,9,PREPARED FOODS,154.553,1,6,NaN,47.57
2778828,2017-08-15,9,PRODUCE,2419.729,148,6,NaN,47.57
2778829,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,6,NaN,47.57


In [31]:
#df_train_merged.to_pickle(df_train.pkl)

## Transformations on test data ##

In [32]:
df_test_transformed = train_to_store_merge(df_test, df_stores)
df_test_transformed = train_to_holiday_merge(df_test_transformed, national_holidays_filtered, state_holidays_filtered, local_holidays_filtered)
df_test_transformed = train_to_oil_merge(df_test_transformed, df_oil)
df_test_transformed = df_test_transformed.drop(['id','city','state', 'type'])

KeyError: "['id', 'city', 'state', 'type'] not found in axis"

In [ ]:
df_test_transformed

In [ ]:
df_dummies = pd.get_dummies(df_test_transformed, columns=['store_nbr','holiday']) ## add family pca here maybe?

In [ ]:
#df_dummies.to_pickle(df_test.pkl)

In [ ]:
#df_test = pd.read_pickle('df_test.pkl')
#df_train = pd.read_pickle('df_train.pkl')